In [ ]:
import torch.nn.functional as F

In [ ]:
# Convert the data into PyTorch datasets and data loaders
train_loader = DataLoader(ProteinDataset(train_data, tokenizer), batch_size=1)
val_loader = DataLoader(ProteinDataset(validation_data, tokenizer), batch_size=1)
test_loader = DataLoader(ProteinDataset(test_data, tokenizer), batch_size=1)

In [ ]:
epoch_no = 10

In [ ]:
# Train the model
for epoch in range(epoch_no):  # 10 epochs
    model.train()
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}')
    for inputs, labels in progress_bar:
        #print(labels)
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs.long()).squeeze()
        loss = criterion(outputs, F.one_hot(labels, num_classes=2).float().view(-1))
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    val_labels = []
    val_preds = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs.long()).squeeze()
            preds = torch.round(torch.sigmoid(outputs))
            max_indices = torch.argmax(preds, dim=0)
            #print(preds)
            #print("------------------")
            val_labels.extend(labels.cpu().numpy())
            val_preds.append(max_indices.cpu().numpy())
    print('Epoch:', epoch, 
          'Validation Accuracy:', accuracy_score(val_labels, val_preds), 
          'Validation F1:', f1_score(val_labels, val_preds), 
          'Validation Precision:', precision_score(val_labels, val_preds), 
          'Validation Recall:', recall_score(val_labels, val_preds))
    progress_bar.close()